In [1]:
import pandas as pd
import numpy as np
filepath='D:/downloads/3405_6663_bundle_archive/'
filename='tmdb_5000_movies.csv'
data=pd.read_csv(filepath+filename)

data.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [2]:
data=data[['id','genres','vote_average','vote_count','popularity','title','keywords','overview']]
tmp_m=data['vote_count'].quantile(0.90)
data=data[data['vote_count']>=tmp_m]

In [3]:
C = data['vote_average'].mean()
M = tmp_m

def weighted_rating(x,m=M,c=C):
    v=x['vote_count']
    R=x['vote_average']
    
    return (v/(v+m)*R) + (m/(m+v)*C)

data['score'] = data.apply(weighted_rating,axis=1)
data.head(2)

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",7.168053
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",6.918271


In [5]:
import ast

data['genres']=data['genres'].apply(ast.literal_eval)
data['keywords']=data['keywords'].apply(ast.literal_eval)

In [7]:
def get_names(element):
    return [info['name'] for info in element]

def join_names(element):
    return ' '.join(element)

data['genres']=data['genres'].apply(get_names).apply(join_names)
data['keywords']=data['keywords'].apply(get_names).apply(join_names)
data[['genres','keywords']].head(2)

,genres,keywords
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...


In [13]:
from sklearn.feature_extraction.text import *

count_vector = CountVectorizer(ngram_range=(1,3))
count_vector.fit(data['genres'])
Genre_matrix = count_vector.transform(data['genres']).toarray()

In [16]:
Genre_matrix.shape

(481, 364)

### __모든 영화 쌍에 대하여 코사인 유사도 계산, 최대 값&인덱스 출력__

In [25]:
from numpy.core.umath_tests import inner1d
def cos_cal(vec1,vec2):
    return np.dot(vec1,vec2)/(np.sqrt(inner1d(vec1,vec1))*np.sqrt(inner1d(vec2,vec2)))

<ipython-input-25-9f0963e7af5b>:1: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [26]:
Movie_cnt=Genre_matrix.shape[0]
Val,idx1,idx2=-1,0,0
for i in range(Movie_cnt):
    for j in range(i+1,Movie_cnt):
        new_val = cos_cal(Genre_matrix[i],Genre_matrix[j])
        if new_val > Val:
            Val,idx1,idx2 = new_val,i,j

print(Val,data['title'][idx1],data['title'][idx2])

1.0000000000000002 Avatar The Lone Ranger
